In [ ]:
#Deconvolution

In [ ]:
import os
os.chdir('/data/kkovacs/Python')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
import scanpy as sc
import copy
import torch
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import gdown
import sankey
import anndata
import pandas as pd

In [ ]:
adata.obs['Decon'] = adata.obs['Level_2_5']
adata.obs['Decon'] = adata.obs['Decon'].astype(object)

adata.X.data = adata.layers['raw'].data.copy()
decon = adata[adata.obs['MLCA_or_query'] == 'Control']
decon.X.data = decon.layers['raw'].data.copy()
batches = np.unique(decon.obs['Decon'])
batches = batches[batches != 'Other cells']
batches = batches.astype(str)
cell_indices = []
for batch in batches:
	idx = np.where(decon.obs['Decon'] == batch)[0]
	np.random.shuffle(idx)
	cell_indices += idx[:200].tolist()

adata_filter = decon[cell_indices,:]

matrix = adata_filter.X.toarray()

mat = pd.DataFrame(data = matrix, index = adata_filter.obs["Decon"],\
	columns = adata_filter.var_names).T

mat.to_csv('Ref_1.tsv',sep = '\t', index_label = 'GeneSymbol')

In [ ]:
Create pseudobulk object from single cell

In [ ]:
# initialize the object
#Needs to be the CPM norm counts for the pseudo
adata.X.data = adata.layers['norm'].data.copy()
adata.obs['Pseudo'] = adata.obs['sample']
adata.obs['Pseudo'] = adata.obs['Pseudo'].astype(object)
#removed due to large proportion of epi cells
adata.obs['Pseudo'].loc[adata.obs['study'] == 'Mouse_Post_Sendai'] = 'Removed'

from adpbulk import ADPBulk
adpb = ADPBulk(adata, "Pseudo")

# perform the pseudobulking
pseudo = adpb.fit_transform().T

pseudo = pseudo.rename(columns=lambda x: x.replace('Pseudo.',''))

pseudo = pseudo.drop('Removed',axis=1)
pseudo.to_csv('/data/kkovacs/Python/Pseudo_Mouse_Atlas.tsv',sep = '\t', index_label='GeneSymbol')  

In [ ]:
#plots post finding chosen reference matrix
data=pd.read_csv('Decon_related/CIBERSORTx_Ref_1.csv',sep=',',index_col='Mixture')
table = pd.crosstab(adata.obs['Pseudo'],adata.obs['Decon'], normalize='index')

data = data.drop(['Correlation','RMSE','P-value'],axis=1)

table.drop('Removed',axis=0,inplace=True)

table = table.drop('Other cells',axis=1)
table = table.add_prefix('Actual_')



num_rows = 2  # Adjust as needed
num_cols = 2  # Adjust as needed
num_plots = int(np.ceil(len(table.columns) / (num_rows * num_cols)))
subplots_per_plot = num_rows * num_cols

for plot_num in range(num_plots):
    start_col = plot_num * subplots_per_plot
    end_col = min((plot_num + 1) * subplots_per_plot, len(table.columns))
    
    # Create a grid of subplots
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 6))
    
    # Flatten the axes array if it's a 2D array
    axes = axes.flatten()
    
    # Loop through the columns and create scatter plots for the current plot
    for i, (col_table, col_data) in enumerate(zip(table.columns[start_col:end_col], data.columns[start_col:end_col])):
        # Select the current subplot
        ax = axes[i]
        
        # Concatenate the columns of interest
        combined_data = pd.concat([table[col_table], data[col_data]], axis=1)
        
        combined_data.plot(x=col_table, y=col_data, kind='scatter', ax=ax, legend=False)
        
        # Calculate and plot the line of best fit
        line = np.polyfit(combined_data[col_table], combined_data[col_data], 1)
        ax.plot(combined_data[col_table], np.polyval(line, combined_data[col_table]), color='red', label=f'Line of Best Fit')
        
        # Set subplot title
        ax.set_title(f'{col_table} vs {col_data}')
        
    # Adjust layout for better spacing
    plt.tight_layout()
    
    # Save the current plot
    plt.savefig(f'scatter_plots_panel_{plot_num + 1}.png')
    plt.clf()